In [1]:
!pip install streamlit tensorflow pandas yfinance matplotlib
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.4 MB/s eta 0:00:00


In [2]:
%%writefile Stock_Price_Prediction_Dashboard.py
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess stock data
def load_data(stock_symbol):
    data = yf.download(stock_symbol, period="5y", interval="1d")
    data = data[['Close']]
    return data

# Prepare data for LSTM model
def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values)

    # Prepare training data
    train_data = scaled_data[:int(len(scaled_data) * 0.8)]
    X_train, y_train = [], []
    for i in range(60, len(train_data)):
        X_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    return X_train, y_train, scaler

# Build LSTM model
def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(60, 1)))
    model.add(tf.keras.layers.LSTM(units=50, return_sequences=False))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Predict stock prices
def predict_prices(model, X_train, scaler, stock_symbol):
    predicted_stock_price = model.predict(X_train)
    predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

    # Load the test data for plotting
    test_data = yf.download(stock_symbol, period="5y", interval="1d")
    actual_stock_price = test_data['Close'].values

    return predicted_stock_price, actual_stock_price

# Streamlit app
st.title('Stock Price Prediction Dashboard')

# User input for stock symbol
stock_symbol = st.text_input('Enter Stock Symbol (e.g. AAPL, MSFT, TSLA):', 'AAPL')

if stock_symbol:
    st.write(f"Showing stock price data for: {stock_symbol}")

    # Load and preprocess stock data
    data = load_data(stock_symbol)

    # Prepare the training data
    X_train, y_train, scaler = prepare_data(data)

    # Build and train the LSTM model
    model = build_model()
    model.fit(X_train, y_train, epochs=1, batch_size=32)

    # Predict stock prices
    predicted_stock_price, actual_stock_price = predict_prices(model, X_train, scaler, stock_symbol)

    # Plot the actual and predicted stock prices
    st.subheader('Stock Price Prediction vs Actual Price')
    plt.figure(figsize=(14, 5))
    plt.plot(actual_stock_price, color='blue', label='Actual Stock Price')
    plt.plot(predicted_stock_price, color='red', label='Predicted Stock Price')
    plt.title(f"{stock_symbol} Stock Price Prediction")
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    st.pyplot(plt)

    st.write("**Stock Prediction Model**")
    st.write(f"This model uses LSTM to predict future stock prices based on the historical stock data of {stock_symbol}.")


Writing Stock_Price_Prediction_Dashboard.py


In [17]:
!ngrok authtoken 2nqFyg1RsjK7UIG3MfnMXUD9FMS_ohZ3SAV651eCJyQW16P5

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
from pyngrok import ngrok
!streamlit run Stock_Price_Prediction_Dashboard.py&>/dev/null&

In [19]:
!pgrep -f streamlit

617
892
1053
1350


In [20]:
public_url=ngrok.connect(8501)

In [7]:
ngrok.kill()

In [21]:
public_url

<NgrokTunnel: "https://cc17-34-125-48-156.ngrok-free.app" -> "http://localhost:8501">